In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [2]:
path = "../WeatherPy/output_data/cities.csv"
weather_df = pd.read_csv(path, index_col=0)
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,New Milford,75,US,1661232703,92,40.9327,-74.0179,75.18,4.61
1,Dikson,100,RU,1661232704,94,73.5069,80.5464,42.13,6.71
2,Maple Creek,2,CA,1661232632,64,49.9168,-109.4848,72.01,10.36
3,Cidreira,3,BR,1661232704,92,-30.1811,-50.2056,59.31,13.91
4,Fortuna,99,US,1661232584,92,40.5982,-124.1573,63.66,1.01
...,...,...,...,...,...,...,...,...,...
562,Cururupu,97,BR,1661232942,93,-1.8283,-44.8683,73.02,3.11
563,Sand,0,DE,1661232942,77,50.0000,10.6000,58.75,2.84
564,Malbork,38,PL,1661232942,85,54.0359,19.0266,70.12,3.62
565,Amahai,83,ID,1661232943,84,-3.3333,128.9167,79.56,7.76


In [3]:
max_humidity = weather_df['Humidity'].max()
max_humidity

100

In [4]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [5]:
locations = weather_df[["Lat", "Lng"]]
weight = weather_df["Humidity"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, 
                                 dissipating = False, max_intensity = max_humidity,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
filter_weather_df = weather_df.loc[(((weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80)) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0) & (weather_df["Country"] == "US")), :]
filter_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
242,Jasper,0,US,1661232789,74,37.2001,-94.3502,71.55,4.61
260,Pascagoula,0,US,1661232815,97,30.3658,-88.5561,77.72,0.00
445,Vidalia,0,US,1661232893,94,32.2177,-82.4135,75.15,0.00
509,Baker City,0,US,1661232919,55,44.7749,-117.8344,71.60,6.91


# Hotel Map

In [8]:
hotel_df = filter_weather_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
242,Jasper,0,US,1661232789,74,37.2001,-94.3502,71.55,4.61,
260,Pascagoula,0,US,1661232815,97,30.3658,-88.5561,77.72,0.00,
445,Vidalia,0,US,1661232893,94,32.2177,-82.4135,75.15,0.00,
509,Baker City,0,US,1661232919,55,44.7749,-117.8344,71.60,6.91,


In [9]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.

params = {"radius": 50000, 
          "types": "hotel", 
          "keyword": "hotel",
          "key": g_key}


In [10]:
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Address,Hotel Rating
242,Jasper,0,US,1661232789,74,37.2001,-94.3502,71.55,4.61,Hampton Inn Miami,"115 Deacon Turner Rd, Miami",4.2
260,Pascagoula,0,US,1661232815,97,30.3658,-88.5561,77.72,0.00,Hampton Inn & Suites Mobile- Downtown Historic...,"62 S Royal St, Mobile",4.6
445,Vidalia,0,US,1661232893,94,32.2177,-82.4135,75.15,0.00,Hampton Inn & Suites Dublin,"103 Travel Center Blvd, Dublin",4.5
509,Baker City,0,US,1661232919,55,44.7749,-117.8344,71.60,6.91,Best Western Sunridge Inn & Conference Center,"1 Sunridge Ln, Baker City",3.9


In [11]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
hotel_list = hotel_df[["City", "Country", "Hotel Name"]]
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))